In [12]:
all_d = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [143]:
def to_four_classes(val):
    if val<=0.25:
        return 0
    if val<=0.5:
        return 1
    if val<=0.75:
        return 2
    else:
        return 3

## Adult

In [42]:
import pandas as pd
adv_df = pd.read_csv("adult/data/shadow_model_outputs_proba-new_test_set.csv")

In [50]:
adult_adv = keras.models.load_model('adult/models/adult_adv_0.72_test_r2.keras')

In [142]:
# measure accuracy
all_out_adult = adult_adv(adv_df.drop(columns="y"))

In [168]:
classification_adult = [to_four_classes(out) == to_four_classes(y) for out, y in zip(all_out_adult, adv_df["y"])]
classification_adult.count(True)/len(classification_adult)

0.3977777777777778

In [46]:
all_adult = []
for d in all_d:
    all_adult.append(adv_df[adv_df["y"]==d])

In [125]:
adult_maes = []
adult_mean_output = []

for adv_test in all_adult:
    adv_y_test = adv_test["y"]
    adv_X_test = adv_test.drop(columns=["y"])
    mae = keras.metrics.MeanAbsoluteError()
    out = adult_adv(adv_X_test)
    m = mae(adv_y_test, out).numpy()
    adult_mean_output.append(np.array(out).mean())
    adult_maes.append(m)

In [126]:
adult_mean_output

[0.29290128,
 0.24803197,
 0.41366032,
 0.51766515,
 0.5921355,
 0.618554,
 0.7270807,
 0.6593976,
 0.65592515]

## UTKFace

In [1]:
import keras

adversary = keras.models.load_model('utkface/models/adv_v2_0.63_r2.keras')

2024-04-27 16:47:18.575258: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-27 16:47:18.580185: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-27 16:47:21.877127: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import pandas as pd

all_adv_test = [pd.read_csv(f"utkface/data/shadow_model_outputs/{dist}/test.csv") for dist in all_d]

In [149]:
# measure accuracy
all_out_utk = adversary(pd.concat(all_adv_test).drop(columns="y"))

In [167]:
classification_utk = [to_four_classes(out) == to_four_classes(y) for out, y in zip(all_out_utk, pd.concat(all_adv_test)["y"])]
classification_utk.count(True)/len(classification_utk)

0.4488888888888889

In [127]:
utk_maes = []
utk_mean_output = []

for adv_test, d in zip(all_adv_test, all_d):
    adv_y_test = adv_test["y"]
    adv_X_test = adv_test.drop(columns=["y"])
    mae = keras.metrics.MeanAbsoluteError()
    out = adversary(adv_X_test)
    m = mae(adv_y_test, out).numpy()
    utk_mean_output.append(np.array(out).mean())
    utk_maes.append(m)

In [132]:
utk_mean_output

[0.27484733,
 0.3760082,
 0.42236623,
 0.47667214,
 0.5431321,
 0.58170485,
 0.5959058,
 0.63321024,
 0.64048094]

In [17]:
utk_maes

[0.17484735,
 0.17600821,
 0.12236618,
 0.085059844,
 0.05526917,
 0.053429887,
 0.10544886,
 0.16678973,
 0.259519]

## CIFAR-10

In [20]:
adv_test_cifar = [pd.read_csv(f"cifar_new_new/data/shadow_model_outputs/{dist}/test.csv") for dist in all_d]

In [22]:
cifar_adv = keras.models.load_model('cifar/models/cifar-adv_0.62_test_r2.keras')

/informatik2/svs/home/stock/anaconda3/envs/pia-tf2/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [151]:
# measure accuracy
all_out_cifar = cifar_adv(pd.concat(adv_test_cifar).drop(columns="y"))

In [166]:
classification_cifar = [to_four_classes(out) == to_four_classes(y) for out, y in zip(all_out_cifar, pd.concat(adv_test_cifar)["y"])]
classification_cifar.count(True) / len(classification_cifar)

0.4922222222222222

In [128]:
cifar_maes = []
cifar_mean_output = []

for adv_test in adv_test_cifar:
    adv_y_test = adv_test["y"]
    adv_X_test = adv_test.drop(columns=["y"])
    out = cifar_adv(adv_X_test)
    mae = keras.metrics.MeanAbsoluteError()
    m = mae(adv_y_test, out).numpy()
    cifar_mean_output.append(np.array(out).mean())
    cifar_maes.append(m)

In [131]:
cifar_mean_output

[0.37891167,
 0.33837262,
 0.3068717,
 0.32162377,
 0.5416798,
 0.5606562,
 0.615737,
 0.6735314,
 0.7246678]

In [32]:
cifar_maes

[0.2789116,
 0.13837264,
 0.044110894,
 0.087120555,
 0.064873174,
 0.061440688,
 0.08833109,
 0.12881975,
 0.17533214]

In [58]:
import pandas as pd

attack_eval = pd.DataFrame({"dist": all_d, "adult": adult_maes, "utk": utk_maes, "cifar": cifar_maes})

In [137]:
attack_eval.to_csv('bb-mean-error-eval.csv', index=False)

In [138]:
attack_eval_wb = pd.DataFrame({"dist": all_d, "adult": adult_mean_output, "utk": utk_mean_output, "cifar": cifar_mean_output})
attack_eval_wb.to_csv('bb-mean-output-eval.csv', index=False)

# White-box
### CIFAR

In [62]:
from common.whitebox_utils import get_wb_meta_clf, do_read_single_model_params
import keras

m_c = keras.models.load_model('cifar/models/shadow_models/0.1/train/0.keras')
adv_input_c = do_read_single_model_params(m_c)

In [63]:
adv_c = get_wb_meta_clf(adv_input_c)

In [64]:
adv_c.load_weights('cifar/models/wb_adv_cifar_0.6_test_r2.weights.h5')

In [108]:
import tensorflow as tf
ds_test = tf.data.Dataset.load("cifar/data/white-box/test-data").batch(10).as_numpy_iterator()

In [109]:
results = []
for x, y in ds_test:
    results.extend(adv_c(x))

2024-04-29 15:16:15.632278: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
split_results = np.array_split(results, len(all_d))

In [157]:
true_y = np.repeat(all_d, len(split_results[0]))

In [165]:
classification_wb_cifar = [to_four_classes(out.numpy())==to_four_classes(y) for out, y in zip(results, true_y)]
classification_wb_cifar.count(True) / len(classification_wb_cifar)

0.6244444444444445

In [110]:
cifar_maes_wb = []
for r, d in zip(split_results, all_d):
    mae = keras.metrics.MeanAbsoluteError()
    m = mae(np.repeat(d, len(r)), r).numpy()
    cifar_maes_wb.append(m)

In [129]:
cifar_wb_mean_output = []
for r in split_results:
    cifar_wb_mean_output.append(np.array(r).mean())

In [130]:
cifar_wb_mean_output

[0.4511328,
 0.38811004,
 0.34906772,
 0.34949586,
 0.35675773,
 0.45573235,
 0.60958815,
 0.7390651,
 0.87239337]

In [111]:
cifar_maes_wb

[0.3511328,
 0.18811007,
 0.05223444,
 0.05121453,
 0.1432423,
 0.14457516,
 0.093326166,
 0.07568662,
 0.053421985]

## UTKFace

In [113]:
utk_base_path = "utkface/models/shadow_models"
m_u = keras.models.load_model(f'{utk_base_path}/0.1/train/0.keras')
adv_input_u = do_read_single_model_params(m_u)

In [115]:
adv_u = get_wb_meta_clf(adv_input_u)

In [116]:
adv_u.load_weights(f"utkface/models/wb_adv_utk_0.86_test_r2.weights.h5")

In [117]:
ds_test_utk = tf.data.Dataset.load("utkface/data/white-box/test-data").batch(8).as_numpy_iterator()

In [118]:
results_u = []
for x, y in ds_test_utk:
    results_u.extend(adv_u(x))

2024-04-29 15:26:35.591444: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [172]:
results_u

[<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0644605], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1379275], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.10558614], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1000615], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1063081], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.11164455], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.09851196], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.08040319], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.13257666], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1037863], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.10100242], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0408865], dtype=float32)>,
 <tf.Tenso

In [ ]:
split_results_u = np.array_split(results_u, len(all_d))

In [174]:
true_y_utk = np.repeat(all_d, len(split_results_u[0]))

In [176]:
classification_wb_utk = [to_four_classes(out.numpy())==to_four_classes(y) for out, y in zip(results_u, true_y_utk)]
classification_wb_utk.count(True) / len(classification_wb_utk)

0.6755555555555556

In [119]:
utk_maes_wb = []
for r, d in zip(split_results_u, all_d):
    mae = keras.metrics.MeanAbsoluteError()
    m = mae(np.repeat(d, len(r)), r).numpy()
    utk_maes_wb.append(m)

In [133]:
utk_wb_mean_output = []
for r in split_results_u:
    utk_wb_mean_output.append(np.array(r).mean())

In [134]:
utk_wb_mean_output

[0.09359901,
 0.21448511,
 0.36983597,
 0.4720241,
 0.5682048,
 0.620875,
 0.6815449,
 0.7226862,
 0.74826056]

In [120]:
utk_maes_wb

[0.025130322,
 0.03922014,
 0.07855163,
 0.08005384,
 0.083696306,
 0.0638503,
 0.06336475,
 0.08502324,
 0.15213047]

In [121]:
attack_eval_wb = pd.DataFrame({"dist": all_d, "utk": utk_maes_wb, "cifar": cifar_maes_wb})

In [136]:
attack_eval_wb.to_csv('wb-mean-error-eval.csv', index=False)

In [135]:
mean_output_wb = pd.DataFrame({"dist": all_d, "utk": utk_wb_mean_output, "cifar": cifar_wb_mean_output})
mean_output_wb.to_csv('wb-mean-output-eval.csv')